In [69]:
# Setting up libraries
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import psycopg2
from config import user, password, hostname

### Connecting to database

In [70]:
# Create engine
engine = create_engine(f'postgres+psycopg2://{user}:{password}@{hostname}/twitter_vs_stocks')

# Use the Inspector to explore the database and print the table names
inspector = inspect(engine)
inspector.get_table_names()

['stock', 'tweets_text', 'twitter_vs_stocks']

In [72]:
twitter_vs_stocks = pd.read_sql_table(
    'twitter_vs_stocks',
    con=engine
)
twitter_vs_stocks.head()

,date,tokenized_text,like_count,quote_count,reply_count,retweet_count,change,volume
0,2021-07-09,"['solar', 'powerwall', 'battery', 'ensures', '...",37454,400,4443,2871,4.140015,18118500
1,2021-07-09,"['tesla', 'solar', 'roof', 'powerwall', 'major...",13972,119,2185,1407,4.140015,18118500
2,2021-07-09,"['autonomous', 'spacex', 'droneship', 'shortfa...",63291,860,3653,6553,4.140015,18118500
3,2021-07-09,"['electrekco', 'bought', 'first', 'tesla', her...",0,0,0,979,4.140015,18118500
4,2021-07-08,"['maybe', 'movie', 'gaslit', 'us']",26485,92,1927,1045,8.159973,22773300


In [ ]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)
Base.classes.keys()

In [33]:
# change name of table -----------------------------<<<<<<<<<<<<<<<<<<
posts_vs_stock = Base.classes.twitter_vs_stocks

AttributeError: twitter_vs_stocks

In [34]:
# Use the Inspector to explore the database and print the table names
inspector = inspect(engine)
inspector.get_table_names()

['stock', 'tweets_text', 'twitter_vs_stocks']

In [63]:
from sqlalchemy import Column, Integer, String, Date, Float
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class twitter_vs_stocks(Base):
    __tablename__ = 'twitter_vs_stocks'
    date = Column(Date)
    tokenized_text = Column(String, primary_key=True)
    like_count = Column(Integer)
    quote_count = Column(Integer)
    reply_count = Column(Integer)
    retweet_count = Column(Integer)
    change = Column(Float)
    volume = Column(Integer)

    def __repr__(self):
        return "(date='{}', tokenized_text='{}', like_count={}, quote_count={}, reply_count={}, retweet_count={}, change={}, volume={})"\
                .format(self.date, self.tokenized_text, self.like_count, self.quote_count, self.reply_count, self.retweet_count, \
                    self.change, self.volume)

In [64]:
# Create  session from Python to the DB
session = Session(engine)

In [65]:
results = session.query(twitter_vs_stocks).first()

In [66]:
results

(date='2021-07-09', tokenized_text='['solar', 'powerwall', 'battery', 'ensures', 'home', 'never', 'loses', 'power']', like_count=37454, quote_count=400, reply_count=4443, retweet_count=2871, change=4.1400146484375, volume=18118500)

In [67]:
df = pd.DataFrame(results, columns=['date','tokenized_text', 'like_count', 'quote_count', 'reply_count', 'retweet_count', 'change', 'volume'])
df.set_index(df['date'], inplace=True)
df.head()

ValueError: DataFrame constructor not properly called!

In [68]:
table_df = pd.read_sql_table(
    'twitter_vs_stocks',
    con=engine
)
table_df.head()

,date,tokenized_text,like_count,quote_count,reply_count,retweet_count,change,volume
0,2021-07-09,"['solar', 'powerwall', 'battery', 'ensures', '...",37454,400,4443,2871,4.140015,18118500
1,2021-07-09,"['tesla', 'solar', 'roof', 'powerwall', 'major...",13972,119,2185,1407,4.140015,18118500
2,2021-07-09,"['autonomous', 'spacex', 'droneship', 'shortfa...",63291,860,3653,6553,4.140015,18118500
3,2021-07-09,"['electrekco', 'bought', 'first', 'tesla', her...",0,0,0,979,4.140015,18118500
4,2021-07-08,"['maybe', 'movie', 'gaslit', 'us']",26485,92,1927,1045,8.159973,22773300


In [ ]:
session.close()